## Day 20: Grove Positioning System

It's finally time to meet back up with the Elves. When you try to contact them, however, you get no reply. Perhaps you're out of range?

You know they're headed to the grove where the star fruit grows, so if you can figure out where that is, you should be able to meet back up with them.

Fortunately, your handheld device has a file (your puzzle input) that contains the grove's coordinates! Unfortunately, the file is encrypted - just in case the device were to fall into the wrong hands.

Maybe you can decrypt it?

When you were still back at the camp, you overheard some Elves talking about coordinate file encryption. The main operation involved in decrypting the file is called mixing.

The encrypted file is a list of numbers. To mix the file, move each number forward or backward in the file a number of positions equal to the value of the number being moved. The list is circular, so moving a number off one end of the list wraps back around to the other end as if the ends were connected.

For example, to move the 1 in a sequence like 4, 5, 6, 1, 7, 8, 9, the 1 moves one position forward: 4, 5, 6, 7, 1, 8, 9. To move the -2 in a sequence like 4, -2, 5, 6, 7, 8, 9, the -2 moves two positions backward, wrapping around: 4, 5, 6, 7, 8, -2, 9.

The numbers should be moved in the order they originally appear in the encrypted file. Numbers moving around during the mixing process do not change the order in which the numbers are moved.

Consider this encrypted file:

```
1
2
-3
3
-2
0
4
```

Mixing this file proceeds as follows:

```
Initial arrangement:
1, 2, -3, 3, -2, 0, 4

1 moves between 2 and -3:
2, 1, -3, 3, -2, 0, 4

2 moves between -3 and 3:
1, -3, 2, 3, -2, 0, 4

-3 moves between -2 and 0:
1, 2, 3, -2, -3, 0, 4

3 moves between 0 and 4:
1, 2, -2, -3, 0, 3, 4

-2 moves between 4 and 1:
1, 2, -3, 0, 3, 4, -2

0 does not move:
1, 2, -3, 0, 3, 4, -2

4 moves between -3 and 0:
1, 2, -3, 4, 0, 3, -2
```

In [1]:
import pytest
from icecream import ic
from typing import Iterable

In [2]:
def load_input(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            line = line.strip()
            yield int(line)

In [3]:
assert list(load_input('sample.txt')) == [
    1,
    2,
    -3,
    3,
    -2,
    0,
    4,
]

> Como tenemos que mover los números por orden, y están repetidos en el problema final, tenemos que poder reconocer si un número se ha movido ya o no. Aquí hacemos la cutrada de volver a ponerlo en la lista en forma de texto. Así, el próximo `index` no  lo localizará. Feo de cojones pero funciona.

In [4]:
def move(numbers: [int], num:int, tron=False) -> [int]:
    size = len(numbers)
    if num == 0:
        if tron:
            print(f"{num} does not move:")
            print(", ".join([str(n) for n in numbers]), end="\n\n")
        return numbers
    pos = numbers.index(num)
    assert num == numbers.pop(pos)
    # ic(f'Now working with the number {num} in {pos}')
    new_pos = (num + pos) % (size - 1)  # Because the pop
    # ic('- moves to', new_pos)
    if tron:
        print(f"{num} moves between", end=' ') 
        if new_pos == 0:
            val_from = numbers[-1]
            val_to = numbers[0]
        elif new_pos == 1:
            val_from = numbers[0]
            val_to = numbers[1]
        elif new_pos == size - 1:
            val_from = numbers[-2]
            val_to = numbers[-1]
        elif new_pos == size:
            val_from = numbers[-1]
            val_to = numbers[0]
        else:
            val_from = numbers[new_pos-1]
            val_to = numbers[new_pos]
        print(f"{val_from} and {val_to} [{pos} -> {new_pos}] ")    
    # ic(pos, new_pos, num)
    if new_pos == 0:
        numbers.append(str(num))   
    else:
        numbers.insert(new_pos, str(num))
    if tron:
        print(", ".join([repr(n) for n in numbers]), end="\n\n")
    return numbers

In [5]:
assert move([2, 1, 7, 4, 5], 2, tron=True) == [1, 7, '2', 4, 5]
assert move([1, 123, 2, 4, 5], 1, tron=True) == [123, '1', 2, 4, 5]
assert move([0, 1, 2, 4, 5], 0, tron=True) == [0, 1, 2, 4, 5]

2 moves between 7 and 4 [0 -> 2] 
1, 7, '2', 4, 5

1 moves between 123 and 2 [0 -> 1] 
123, '1', 2, 4, 5

0 does not move:
0, 1, 2, 4, 5



In [6]:
assert move([-1, 0, 1, 2, 4, 5], -1) == [0, 1, 2, 4, '-1', 5]
assert move([-2, 0, 1, 2, 4, 5], -2) == [0, 1, 2, '-2', 4, 5]
assert move([-3, 0, 1, 2, 4, 5], -3) == [0, 1, '-3', 2, 4, 5]
assert move([-4, 0, 1, 2, 4, 5], -4) == [0, '-4', 1, 2, 4, 5]
assert move([-5, 0, 1, 2, 4, 5], -5) == [0, 1, 2, 4, 5, '-5']

assert move([-6, 0, 1, 2, 4, 5], -6) == [0, 1, 2, 4, '-6', 5]
assert move([-7, 0, 1, 2, 4, 5], -7) == [0, 1, 2, '-7', 4, 5]
assert move([-8, 0, 1, 2, 4, 5], -8) == [0, 1, '-8', 2, 4, 5]
assert move([-9, 0, 1, 2, 4, 5], -9) == [0, '-9', 1, 2, 4, 5]
assert move([-10, 0, 1, 2, 4, 5], -10) == [0, 1, 2, 4, 5, '-10']
assert move([-11, 0, 1, 2, 4, 5], -11) == [0, 1, 2, 4, '-11', 5]

In [7]:
source = load_input('input.txt')  
numbers = list(source)
num = numbers[0]

assert num == -1235
assert numbers[0] == num
assert numbers[159] == num
assert numbers[3091] == num
numbers = move(numbers, num) 
assert int(numbers[158]) == num
assert int(numbers[3090]) == num
assert int(numbers[3764]) == num


In [8]:
# Testing negatives

assert move([1, 4, -2, 5, 6, 7, 8, 9], -2) == [1, 4, 5, 6, 7, 8, 9, '-2']
assert move([4, -2, 5, 6, 7, 8, 9], -2) == [4, 5, 6, 7, 8, '-2', 9]
assert move([0, 1, 4, -2, 5, 6, 7, 8, 9], -2) == [0, '-2', 1, 4, 5, 6, 7, 8, 9]

In [9]:
# Testing ppsitive numbers

assert move([4, 7, 5, 2, 3, 8, 9], 2) == [4, 7, 5, 3, 8, '2', 9]
assert move([4, 9, 7, 5, 2, 3, 8], 2) == [4, 9, 7, 5, 3, 8, '2']
assert move([4, 9, 7, 8, 5, 2, 3], 2) == [4, '2', 9, 7, 8, 5, 3]
assert move([4, 9, 7, 8, 5, 3, 2], 2) == [4, 9, '2', 7, 8, 5, 3]

In [10]:
# Testing repeated numbers

assert move([1, 4, 9, 7, 1, 5, 1, 2], 1) == [4, '1', 9, 7, 1, 5, 1, 2]
assert move([4, '1', 9, 7, 1, 5, 1, 2], 1) == [4, '1', 9, 7, 5, '1', 1, 2]
assert move([4, '1', 9, 7, 5, '1', 1, 2], 1) == [4, '1', 9, 7, 5, '1', 2, '1']
assert move([-1, 4, 9, 7, -1, 5, 1, 2], -1, tron=True) == [4, 9, 7, -1, 5, 1, '-1', 2]

-1 moves between 1 and 2 [0 -> 6] 
4, 9, 7, -1, 5, 1, '-1', 2



In [11]:
def solution_one(source: Iterable, tron=False):
    code = list(iter(source))
    assert 0 in code
    size = len(code)
    if tron:
        print("Initial arrangement:")
        print(", ".join([str(n) for n in code]), end="\n\n")
    for num in code[:]:
        code = move(code, num, tron=tron)
    zero_pos = code.index(0) 
    after_1k = int(code[(zero_pos + 1000) % size])
    after_2k = int(code[(zero_pos + 2000) % size])
    after_3k = int(code[(zero_pos + 3000) % size])
    print(f"{after_1k} + {after_2k} + {after_3k} = {after_1k + after_2k + after_3k}")
    return (after_1k + after_2k + after_3k)    

In [12]:
assert solution_one(load_input('sample.txt'), tron=True) == 3

Initial arrangement:
1, 2, -3, 3, -2, 0, 4

1 moves between 2 and -3 [0 -> 1] 
2, '1', -3, 3, -2, 0, 4

2 moves between -3 and 3 [0 -> 2] 
'1', -3, '2', 3, -2, 0, 4

-3 moves between -2 and 0 [1 -> 4] 
'1', '2', 3, -2, '-3', 0, 4

3 moves between 0 and 4 [2 -> 5] 
'1', '2', -2, '-3', 0, '3', 4

-2 moves between 4 and 1 [2 -> 0] 
'1', '2', '-3', 0, '3', 4, '-2'

0 does not move:
1, 2, -3, 0, 3, 4, -2

4 moves between -3 and 0 [5 -> 3] 
'1', '2', '-3', '4', 0, '3', '-2'

4 + -3 + 2 = 3


In [13]:
solution_one(load_input('test.txt'), tron=True)

Initial arrangement:
1, 0, 2, 10, 4, -3, -2, 3, -1, -4

1 moves between 0 and 2 [0 -> 1] 
0, '1', 2, 10, 4, -3, -2, 3, -1, -4

0 does not move:
0, 1, 2, 10, 4, -3, -2, 3, -1, -4

2 moves between 4 and -3 [2 -> 4] 
0, '1', 10, 4, '2', -3, -2, 3, -1, -4

10 moves between 4 and 2 [2 -> 3] 
0, '1', 4, '10', '2', -3, -2, 3, -1, -4

4 moves between -2 and 3 [2 -> 6] 
0, '1', '10', '2', -3, -2, '4', 3, -1, -4

-3 moves between 0 and 1 [4 -> 1] 
0, '-3', '1', '10', '2', -2, '4', 3, -1, -4

-2 moves between 1 and 10 [5 -> 3] 
0, '-3', '1', '-2', '10', '2', '4', 3, -1, -4

3 moves between 0 and -3 [7 -> 1] 
0, '3', '-3', '1', '-2', '10', '2', '4', -1, -4

-1 moves between 2 and 4 [8 -> 7] 
0, '3', '-3', '1', '-2', '10', '2', '-1', '4', -4

-4 moves between -2 and 10 [9 -> 5] 
0, '3', '-3', '1', '-2', '-4', '10', '2', '-1', '4'

0 + 0 + 0 = 0


0

Then, the grove coordinates can be found by looking at the **1000th**, **2000th**, and **3000th** numbers after the value `0`, wrapping around the list as necessary. In the above example, the 1000th number after `0` is `4`, the 2000th is `-3`, and the 3000th is `2`; adding these together produces **3**.

In [14]:
assert solution_one(load_input('sample.txt')) == 3

4 + -3 + 2 = 3


**Mix your encrypted file exactly once. What is the sum of the three numbers that form the grove coordinates?**

In [15]:
sol = solution_one(load_input('input.txt'), tron=False)
print(f"Solution part one: {sol}")

6076 + 9867 + 590 = 16533
Solution part one: 16533


## Parte 2

The grove coordinate values seem nonsensical. While you ponder the mysteries of Elf encryption, you suddenly remember the rest of the decryption routine you overheard back at camp.

First, you need to apply the decryption key, 811589153. Multiply each number by the decryption key before you begin; this will produce the actual list of numbers to mix.

Second, you need to mix the list of numbers ten times. The order in which the numbers are mixed does not change during mixing; the numbers are still moved in the order they appeared in the original, pre-mixed list. (So, if -3 appears fourth in the original list of numbers to mix, -3 will be the fourth number to move during each round of mixing.)

Using the same example as above:

```
Initial arrangement:
811589153, 1623178306, -2434767459, 2434767459, -1623178306, 0, 3246356612

After 1 round of mixing:
0, -2434767459, 3246356612, -1623178306, 2434767459, 1623178306, 811589153

After 2 rounds of mixing:
0, 2434767459, 1623178306, 3246356612, -2434767459, -1623178306, 811589153

After 3 rounds of mixing:
0, 811589153, 2434767459, 3246356612, 1623178306, -1623178306, -2434767459

After 4 rounds of mixing:
0, 1623178306, -2434767459, 811589153, 2434767459, 3246356612, -1623178306

After 5 rounds of mixing:
0, 811589153, -1623178306, 1623178306, -2434767459, 3246356612, 2434767459

After 6 rounds of mixing:
0, 811589153, -1623178306, 3246356612, -2434767459, 1623178306, 2434767459

After 7 rounds of mixing:
0, -2434767459, 2434767459, 1623178306, -1623178306, 811589153, 3246356612

After 8 rounds of mixing:
0, 1623178306, 3246356612, 811589153, -2434767459, 2434767459, -1623178306

After 9 rounds of mixing:
0, 811589153, 1623178306, -2434767459, 3246356612, 2434767459, -1623178306

After 10 rounds of mixing:
0, -2434767459, 1623178306, 3246356612, -1623178306, 2434767459, 811589153
```

> Ya no nos vale la cutrada de la parte 1, tenemos que hacerlo bien. Usaremos tuplas (índice, número) para poder identificar cada número de forma independiente.

In [16]:
def load_input(filename, key=1):
    with open(filename, 'r') as f_input:
        for i, line in enumerate(f_input):
            line = line.strip()
            yield i, int(line) * key

In [17]:
source = list(load_input('sample.txt', key=811589153))

In [18]:
assert source == [
    (0, 811589153),
    (1, 1623178306),
    (2, -2434767459),
    (3, 2434767459),
    (4, -1623178306),
    (5, 0),
    (6, 3246356612),
]

In [19]:
def move_two(numbers: [tuple[int, int]], duple: tuple[int, int], tron=False) -> [tuple[int, int]]:
    size = len(numbers)
    idx, num = duple
    if num == 0:
        if tron:
            print(f"{num} does not move:")
            print(", ".join([str(n) for n in numbers]), end="\n\n")
        return numbers
    pos = numbers.index(duple)
    (idx, num) = target = numbers.pop(pos)
    new_pos = (num + pos) % (size - 1)  # Because the pop
    # ic('- moves to', new_pos)
    if tron:
        print(f"{num} moves between", end=' ') 
        if new_pos == 0:
            val_from = numbers[-1][1]
            val_to = numbers[0][1]
        elif new_pos == 1:
            val_from = numbers[0][1]
            val_to = numbers[1][1]
        elif new_pos == size - 1:
            val_from = numbers[-2][1]
            val_to = numbers[-1][1]
        elif new_pos == size:
            val_from = numbers[-1][1]
            val_to = numbers[0][1]
        else:
            val_from = numbers[new_pos-1][1]
            val_to = numbers[new_pos][1]
        print(f"{val_from} and {val_to} [{pos} -> {new_pos}] ")    
    # ic(pos, new_pos, num)
    if new_pos == 0:
        numbers.append(duple)   
    else:
        numbers.insert(new_pos, duple)
    if tron:
        print(", ".join([repr(n) for n in numbers]), end="\n\n")
    return numbers

In [20]:
def solution_two(source: Iterable, tron=False):
    code = list(iter(source))
    size = len(code)
    order = code[:]
    if tron:
        print("Initial arrangement:")
        print(", ".join([str(n[1]) for n in code]), end="\n\n")
    for r in range(10):
        for duple in order:
            code = move_two(code, duple)
        if tron:
            print(f"After {r+1} round of mixing:")
            print(*[x[1] for x in code], sep=', ')
            print()
    for i, (_, val) in enumerate(code):
        if val == 0:
            zero_pos = i
            break
    after_1k = int(code[(zero_pos + 1000) % size][1])
    after_2k = int(code[(zero_pos + 2000) % size][1])
    after_3k = int(code[(zero_pos + 3000) % size][1])
    print(f"{after_1k} + {after_2k} + {after_3k} = {after_1k + after_2k + after_3k}")
    return (after_1k + after_2k + after_3k)

In [21]:
sol = solution_two(load_input('sample.txt', key=811589153), tron=True)
sol

Initial arrangement:
811589153, 1623178306, -2434767459, 2434767459, -1623178306, 0, 3246356612

After 1 round of mixing:
0, -2434767459, 3246356612, -1623178306, 2434767459, 1623178306, 811589153

After 2 round of mixing:
0, 2434767459, 1623178306, 3246356612, -2434767459, -1623178306, 811589153

After 3 round of mixing:
0, 811589153, 2434767459, 3246356612, 1623178306, -1623178306, -2434767459

After 4 round of mixing:
0, 1623178306, -2434767459, 811589153, 2434767459, 3246356612, -1623178306

After 5 round of mixing:
0, 811589153, -1623178306, 1623178306, -2434767459, 3246356612, 2434767459

After 6 round of mixing:
0, 811589153, -1623178306, 3246356612, -2434767459, 1623178306, 2434767459

After 7 round of mixing:
0, -2434767459, 2434767459, 1623178306, -1623178306, 811589153, 3246356612

After 8 round of mixing:
0, 1623178306, 3246356612, 811589153, -2434767459, 2434767459, -1623178306

After 9 round of mixing:
0, 811589153, 1623178306, -2434767459, 3246356612, 2434767459, -162317

1623178306

The grove coordinates can still be found in the same way. Here, the 1000th number after 0 is 811589153, the 2000th is 2434767459, and the 3000th is -1623178306; adding these together produces **1623178306**.

**Apply the decryption key and mix your encrypted file ten times. What is the sum of the three numbers that form the grove coordinates?**

In [22]:
assert solution_two(load_input('sample.txt', key=811589153)) == 1623178306

811589153 + 2434767459 + -1623178306 = 1623178306


In [23]:
sol = solution_two(load_input('input.txt', key=811589153))
print(f"Solution part two: {sol}")

-4636608831089 + 7301867609541 + 2124740402554 = 4789999181006
Solution part two: 4789999181006
